In [1]:
import hddm
from patsy import dmatrix  # for generation of (regression) design matrices
import numpy as np         # for basic matrix operations
from pandas import Series  # to manipulate data-frames generated by hddm

In [2]:
import sys
sys.stdout = open('ModelRecoveryOutput.txt', 'w')

## Creating simulated data for the experiment

In [3]:
n_subjects = 10
trials_per_level = 150 # and per stimulus

In [4]:
level1a = {'v':.3, 'a':2, 't':.3, 'sv':0, 'z':.5, 'sz':0, 'st':0}
level2a = {'v':.4, 'a':2, 't':.3, 'sv':0, 'z':.6, 'sz':0, 'st':0}
level3a = {'v':.5, 'a':2, 't':.3, 'sv':0, 'z':.7, 'sz':0, 'st':0}

In [5]:
data_a, params_a = hddm.generate.gen_rand_data({'level1': level1a,
                                                'level2': level2a,
                                                'level3': level3a},
                                                size=trials_per_level,
                                                subjs=n_subjects)

In [6]:
level1b = {'v':.3, 'a':2, 't':.3,'sv': 0, 'z':.5, 'sz': 0, 'st': 0}
level2b = {'v':.4, 'a':2, 't':.3,'sv': 0, 'z':.4, 'sz': 0, 'st': 0}
level3b = {'v':.5, 'a':2, 't':.3,'sv': 0, 'z':.3, 'sz': 0, 'st': 0}

In [7]:
data_b, params_b = hddm.generate.gen_rand_data({'level1': level1b,
                                                'level2': level2b,
                                                'level3': level3b},
                                                size=trials_per_level,
                                                subjs=n_subjects)

In [8]:
data_a['stimulus'] = Series(np.ones((len(data_a))), index=data_a.index)
data_b['stimulus'] = Series(np.ones((len(data_b)))*2, index=data_a.index)

In [9]:
mydata = data_a.append(data_b, ignore_index=True)

In [10]:
mydata

,rt,response,subj_idx,condition,stimulus
0,3.294360,1.0,0,level1,1.0
1,2.726824,0.0,0,level1,1.0
2,1.322725,0.0,0,level1,1.0
3,1.441071,0.0,0,level1,1.0
4,0.308506,0.0,0,level1,1.0
...,...,...,...,...,...
8995,1.143472,1.0,9,level3,2.0
8996,1.573880,1.0,9,level3,2.0
8997,1.414023,1.0,9,level3,2.0
8998,2.281450,1.0,9,level3,2.0


## Setting up the HDDM regression model

In [13]:
def z_link_func(x, data=mydata):
    stim = (np.asarray(dmatrix('0 + C(s, [[0], [1]])',
                              {'s': data.stimulus.loc[x.index]}))
    )
    # Apply z = (1 - x) to flip them along 0.5
    z_flip = stim - x
    # The above inverts those values we do not want to flip,
    # so invert them back
    z_flip[stim == 0] *= -1
    return z_flip

In [19]:
def z_link_func(x, data=mydata):
    stim = (np.asarray(dmatrix('0 + C(s, [[0], [1]])',
                              {'s': data.stimulus.loc[x.index]},return_type='dataframe'))
    )
    # Apply z = (1 - x) to flip them along 0.5
    z_flip = np.subtract(stim, x.to_frame())
    # The above inverts those values we do not want to flip,
    # so invert them back
    z_flip[stim == 0] *= -1
    return z_flip

In [20]:
z_reg = {'model': 'z ~ 1 + C(condition)', 'link_func': z_link_func}

In [15]:
v_reg = {'model': 'v ~ 1 + C(condition)', 'link_func': lambda x: x}

In [23]:
reg_descr = [z_reg, v_reg]

In [24]:
m_reg = hddm.HDDMRegressor(mydata, reg_descr, include=['v', 'a', 't', 'z'])

In [ ]:
m_reg.sample(5000, burn=200)

In [ ]:
m_reg.print_stats()